In [ ]:
# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
import folium  # for geospatial visualization

In [ ]:
# Load and Visualize Sample Data

In [ ]:
# Example tabular climate data (NOAA or Kaggle subset)
climate_data = pd.read_csv("climate_data_subset.csv")
print(climate_data.head())

# Basic visualization of climate trends
plt.figure(figsize=(10,5))
sns.lineplot(data=climate_data, x='date', y='temperature')
plt.title("Sample Temperature Trend")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.show()

In [ ]:
# Preprocess Tabular Data

In [ ]:
# Handle missing values, scaling, encoding categorical features
climate_data.fillna(method='ffill', inplace=True)
features = climate_data.drop(['target_variable', 'date'], axis=1)
target = climate_data['target_variable']

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train_tab, X_test_tab, y_train, y_test = train_test_split(
    features_scaled, target, test_size=0.2, random_state=42
)


In [ ]:
# Load Sample Satellite Images

In [ ]:
img_dir = "satellite_images/"
img_size = (128,128)

def load_images(img_dir, img_size):
    imgs = []
    for fname in os.listdir(img_dir):
        if fname.endswith(".png") or fname.endswith(".jpg"):
            img_path = os.path.join(img_dir, fname)
            img_array = image.load_img(img_path, target_size=img_size)
            img_array = image.img_to_array(img_array)/255.0
            imgs.append(img_array)
    return np.array(imgs)

X_images = load_images(img_dir, img_size)
print("Satellite images shape:", X_images.shape)

In [ ]:
# Define CNN for Image Feature Extraction

In [ ]:
cnn_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128,128,3))
cnn_base.trainable = False  # freeze base for transfer learning

cnn_model = models.Sequential([
    cnn_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu')
])

In [ ]:
# Combine Tabular + Image Data (Multimodal Fusion)

In [ ]:
# Tabular input
input_tab = layers.Input(shape=(X_train_tab.shape[1],))
x_tab = layers.Dense(128, activation='relu')(input_tab)
x_tab = layers.Dropout(0.2)(x_tab)

# Image input
input_img = layers.Input(shape=(128,128,3))
x_img = cnn_model(input_img)

# Fusion
combined = layers.Concatenate()([x_tab, x_img])
x = layers.Dense(256, activation='relu')(combined)
x = layers.Dropout(0.3)(x)
output = layers.Dense(1, activation='linear')(x)  # regression for climate impact variable

multimodal_model = tf.keras.Model(inputs=[input_tab, input_img], outputs=output)

multimodal_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Train Model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

history = multimodal_model.fit(
    [X_train_tab, X_images], y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16,
    callbacks=[early_stop, reduce_lr]
)